In [2]:
import pandas as pd
import numpy as np

In [4]:
bb = pd.read_csv('./data/billboard.csv')
bb.head(5)

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# First unpivot week columns into week, rating (cols => rows)
bb_melt = bb.melt(
            id_vars=['year', 'artist', 'track', 'time', 'date.entered']
            , value_vars=None
            , var_name='week'
            , value_name='rating'
        )
bb_melt[bb_melt.track == 'Loser'].head(5)

,year,artist,track,time,date.entered,week,rating
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
320,2000,3 Doors Down,Loser,4:24,2000-10-21,wk2,76.0
637,2000,3 Doors Down,Loser,4:24,2000-10-21,wk3,72.0
954,2000,3 Doors Down,Loser,4:24,2000-10-21,wk4,69.0
1271,2000,3 Doors Down,Loser,4:24,2000-10-21,wk5,67.0


# 3rd Normal Form

In [7]:
# Subset out songs
songs = bb_melt.loc[:, ['year', 'artist', 'track', 'time']]
songs[songs.track == 'Loser'].head(5)

,year,artist,track,time
3,2000,3 Doors Down,Loser,4:24
320,2000,3 Doors Down,Loser,4:24
637,2000,3 Doors Down,Loser,4:24
954,2000,3 Doors Down,Loser,4:24
1271,2000,3 Doors Down,Loser,4:24


In [8]:
# Take distinct
songs = songs.drop_duplicates()
songs[songs.track == 'Loser']

,year,artist,track,time
3,2000,3 Doors Down,Loser,4:24


In [9]:
# Assign primary key
songs['id'] = songs.index # or 
#songs['id2'] = range(songs.shape[0])
songs.head(5)

,year,artist,track,time,id
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,1
2,2000,3 Doors Down,Kryptonite,3:53,2
3,2000,3 Doors Down,Loser,4:24,3
4,2000,504 Boyz,Wobble Wobble,3:35,4


# Joining

In [12]:
# Inner join to bring in song id
ratings = bb_melt.merge(
            right=songs
            , how='inner'
            , on=['year', 'artist', 'track', 'time']
        )

ratings = ratings.loc[ratings.track == 'Loser', ['id', 'time', 'date.entered', 'week', 'rating', 'track']]
ratings.head(5)

,id,time,date.entered,week,rating,track
228,3,4:24,2000-10-21,wk1,76.0,Loser
229,3,4:24,2000-10-21,wk2,76.0,Loser
230,3,4:24,2000-10-21,wk3,72.0,Loser
231,3,4:24,2000-10-21,wk4,69.0,Loser
232,3,4:24,2000-10-21,wk5,67.0,Loser


In [13]:
site = pd.read_csv('./data/survey_site.csv')
visited = pd.read_csv('./data/survey_visited.csv')

In [14]:
# Join visited to site for lat/lon
v2s = visited.merge(
            right=site
            , how='left'
            , left_on=['site']
            , right_on=['name']
            , indicator=True
        )

v2s.head(5)

,ident,site,dated,name,lat,long,_merge
0,619,DR-1,1927-02-08,DR-1,-49.85,-128.57,both
1,622,DR-1,1927-02-10,DR-1,-49.85,-128.57,both
2,734,DR-3,1939-01-07,DR-3,-47.15,-126.72,both
3,735,DR-3,1930-01-12,DR-3,-47.15,-126.72,both
4,751,DR-3,1930-02-26,DR-3,-47.15,-126.72,both
